In [3]:
import pandas as pd
import joblib
import warnings
import os

# --- 1. Load All Necessary Components ---

try:
    # Load the "recipe" and the feature list
    model_payload = joblib.load('models/preprocessor_and_features.joblib')
    preprocessor = model_payload['preprocessor']
    features_list = model_payload['feature_list']
    
    # Load the "brain" (K-Means model)
    model = joblib.load('models/kmeans_model.joblib')
    
    # Load the "food database"
    food_df = pd.read_csv('data/nutrients_readable_cleaned.csv')
    
    print(f"Models and data loaded successfully. Using {len(features_list)} features.")
    print("Model engine: K-Means (Champion Model, k=6)")

except FileNotFoundError as e:
    print(f"ERROR: Could not load models or data. File not found: {e}")
    print("Please run 'clean_and_explore.py' and 'train_model.py' first.")
    preprocessor, model, food_df, features_list = None, None, None, []
except Exception as e:
    print(f"An error occurred during loading: {e}")
    preprocessor, model, food_df, features_list = None, None, None, []


def recommend_diet(new_user_data):
    """
    Recommends a diet and foods for a new user using the trained K-Means.
    """
    
    if food_df is None or preprocessor is None or model is None:
        print("ERROR: System is not ready. Cannot make recommendation.")
        return None

    user_df = pd.DataFrame([new_user_data])
    
    try:
        user_df = user_df[features_list] 
    except KeyError as e:
        print(f"Error: Missing feature in new user data: {e}")
        return None
    
    # 1. Preprocess the new user's data
    user_processed = preprocessor.transform(user_df)
    
    # 2. Predict the cluster
    user_cluster = model.predict(user_processed)[0]
    
    # 3. Define diet logic and recommend foods
    
    # ======================================================================
    # !!! This logic is now UPDATED based on your cluster analysis !!!
    # ======================================================================
    
    profile_name = ""
    recommendation_type = ""
    recommended_foods = None
    
    print("\n" + "="*30)
    print("--- RECOMMENDATION RESULT ---")
    print(f"Assigned User to Cluster: {user_cluster}")
    
    if user_cluster == 0:
        profile_name = "Prediabetic, Active Profile"
        recommendation_type = "Low-Carb Foods"
        recommended_foods = food_df.sort_values(by='Carbs', ascending=True).head(10)
        
    elif user_cluster == 1:
        profile_name = "Older, Active Diabetic Profile"
        recommendation_type = "Low-Carb Foods"
        recommended_foods = food_df.sort_values(by='Carbs', ascending=True).head(10)
        
    elif user_cluster == 2:
        profile_name = "Active Diabetic with High Blood Pressure"
        recommendation_type = "Low-Fat Foods"
        recommended_foods = food_df.sort_values(by='Fat', ascending=True).head(10)
        
    elif user_cluster == 3:
        profile_name = "Low-Activity Diabetic with Very High Blood Sugar"
        recommendation_type = "Low-Carb Foods"
        recommended_foods = food_df.sort_values(by='Carbs', ascending=True).head(10)
    
    elif user_cluster == 4:
        profile_name = "Active Walker with High Blood Pressure"
        recommendation_type = "Low-Fat Foods"
        recommended_foods = food_df.sort_values(by='Fat', ascending=True).head(10)

    elif user_cluster == 5:
        profile_name = "High BMI (Obese) with Diabetes"
        recommendation_type = "Balanced (Low-Calorie) Foods"
        recommended_foods = food_df.sort_values(by='Calories', ascending=True).head(10)
        
    else:
        # Fallback just in case
        profile_name = f"General Profile (Cluster {user_cluster})"
        recommendation_type = "Balanced (Low-Calorie) Foods"
        recommended_foods = food_df.sort_values(by='Calories', ascending=True).head(10)

    # 4. Format the output
    print(f"Assigned Diet Profile: {profile_name}")
    print(f"\nRecommendation: Top 10 {recommendation_type}")
    
    return recommended_foods[['Food', 'Category', 'Protein', 'Fat', 'Carbs', 'Calories']]


# --- 4. Test the Complete System ---
if __name__ == "__main__":
    warnings.filterwarnings('ignore')
    
    if preprocessor is not None:
        
        # Test User 1 (High BMI, High Sugar, Low Steps)
        # Should match Cluster 3 or 5
        sample_user_1 = {
            'Age': 45, 'Gender': 'Male', 'BMI': 32.0, 'Chronic_Disease': 'Diabetes',
            'Blood_Pressure_Systolic': 140, 'Blood_Pressure_Diastolic': 90,
            'Cholesterol_Level': 210, 'Blood_Sugar_Level': 180, 'Genetic_Risk_Factor': 'Yes',
            'Allergies': 'None', 'Daily_Steps': 3500, 'Exercise_Frequency': 1,
            'Sleep_Hours': 7.0, 'Alcohol_Consumption': 'No', 'Smoking_Habit': 'No',
            'Dietary_Habits': 'Regular', 'Preferred_Cuisine': 'Western', 
            'Food_Aversions': 'None'
        }
        
        recommendation_1 = recommend_diet(sample_user_1)
        if recommendation_1 is not None:
            print(recommendation_1)

        # Test User 2 (Healthy, Active)
        # Should match Cluster 0
        sample_user_2 = {
            'Age': 30, 'Gender': 'Female', 'BMI': 22.0, 'Chronic_Disease': 'None',
            'Blood_Pressure_Systolic': 110, 'Blood_Pressure_Diastolic': 70,
            'Cholesterol_Level': 180, 'Blood_Sugar_Level': 90, 'Genetic_Risk_Factor': 'No',
            'Allergies': 'None', 'Daily_Steps': 11000, 'Exercise_Frequency': 5,
            'Sleep_Hours': 8.0, 'Alcohol_Consumption': 'No', 'Smoking_Habit': 'No',
            'Dietary_Habits': 'Regular', 'Preferred_Cuisine': 'Mediterranean', 'Food_Aversions': 'None'
        }
        
        recommendation_2 = recommend_diet(sample_user_2)
        if recommendation_2 is not None:
            print(recommendation_2)

Models and data loaded successfully. Using 10 features.
Model engine: K-Means (Champion Model, k=6)

--- RECOMMENDATION RESULT ---
Assigned User to Cluster: 3
Assigned Diet Profile: Low-Activity Diabetic with Very High Blood Sugar

Recommendation: Top 10 Low-Carb Foods
                   Food                 Category  Protein   Fat  Carbs  \
69               Turkey            Meat, Poultry       27  15.0    0.0   
34            Margarine  Fats, Oils, Shortenings        0  91.0    0.0   
35  Margarine, 2 pat or  Fats, Oils, Shortenings        0  11.0    0.0   
36           Mayonnaise  Fats, Oils, Shortenings        0  12.0    0.0   
37             Corn oil  Fats, Oils, Shortenings        0  14.0    0.0   
38            Olive oil  Fats, Oils, Shortenings        0  14.0    0.0   
39   Safflower seed oil  Fats, Oils, Shortenings        0  14.0    0.0   
73                  Cod            Fish, Seafood       28   5.0    0.0   
71                Roast            Meat, Poultry       13  14.0 